In [46]:
import pandas as pd

# Extraction

dataFrame = pd.read_excel("./dataset_account.xlsx", engine="openpyxl")

dataFrameProduct = pd.read_excel(
    "./dataset_account.xlsx", sheet_name="product mapping", engine="openpyxl"
)
dataFrameBranch = pd.read_excel(
    "./dataset_account.xlsx", sheet_name="branch mapping", engine="openpyxl"
)

dataFrame["BankCode"] = dataFrame["noCompte"].astype(str).str[0:5]
dataFrame["BranchCode"] = dataFrame["noCompte"].astype(str).str[6:11]
dataFrame["ProductCode"] = dataFrame["noCompte"].astype(str).str[12]
dataFrame["AccountNumber"] = dataFrame["noCompte"].astype(str).str[11:19]


def format(branchCode):
    return ("00000000000000000000000" + str(branchCode))[-5:]


dataFrameBranch["CodeBranch"] = dataFrameBranch.apply(lambda row: format(row["CodeBranch"]), axis=1)


dataFrame["BranchCode"] = dataFrame["BranchCode"].astype(str)
dataFrameBranch["CodeBranch"] = dataFrameBranch["CodeBranch"].astype(str)
dataFrame["BranchCode"] = dataFrame["BranchCode"].astype(str)
dataFrameProduct["productCode"] = dataFrameProduct["productCode"].astype(str)
dataFrame["ProductCode"] = dataFrame["ProductCode"].astype(str)


dataFrameMerged = pd.merge(
    dataFrame, dataFrameProduct, left_on="ProductCode", right_on="productCode", how="left"
)

dataFrameMerged = pd.merge(
    dataFrameMerged, dataFrameBranch, left_on="BranchCode", right_on="CodeBranch", how="left"
)

#dataFrameMerged = dataFrameMerged[dataFrameMerged["AccountStatus"] == "Actif"]

dataFrameMerged["OpeningDate"] = pd.to_datetime(dataFrameMerged["OpeningDate"])
dataFrameMerged["Report_date_to"] = pd.to_datetime(dataFrameMerged["Report_date_to"])
dataFrameMerged["AvailableBalance"] = pd.to_numeric(dataFrameMerged["AvailableBalance"])


dataFrameMerged

,Code,noCompte,AvailableBalance,OpeningDate,AccountStatus,Report_date_to,gestionnaire de compte,BankCode,BranchCode,ProductCode,AccountNumber,productCode,Product,CodeBranch,Branch
0,CLNT-00005640,00115 00001 90000316601 39,1.698926e+09,2024-12-30,Active,2024-12-31,Yanis Blanchard,00115,00001,9,9000031,9,Dépôt à Terme,00001,Champagne-Ardenne
1,CLNT-00022135,00115 00004 90000316600 53,1.322200e+09,2024-12-06,Active,2024-12-31,Sergei Titov,00115,00004,9,9000031,9,Dépôt à Terme,00004,Bourgogne
2,CLNT-00084246,00115 00014 80000316599 27,1.117828e+09,2022-05-23,Active,2024-12-31,Samuel Otieno,00115,00014,8,8000031,8,Compte Epargne,00014,Franche-Comté
3,CLNT-00005629,00115 00001 80000316598 31,1.061136e+09,2023-11-28,Active,2024-12-31,Enzo Giraud,00115,00001,8,8000031,8,Compte Epargne,00001,Champagne-Ardenne
4,CLNT-00005639,00115 00001 90000316597 53,1.000000e+09,2024-12-10,Active,2024-12-31,Enzo Giraud,00115,00001,9,9000031,9,Dépôt à Terme,00001,Champagne-Ardenne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316596,CLNT-00096913,00115 02001 80000000009 46,0.000000e+00,2022-05-04,Active,2024-12-31,Timéo Guillet,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais
316597,CLNT-00096916,00115 02001 80000000012 25,0.000000e+00,2024-02-21,Active,2024-12-31,Timéo Guillet,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais
316598,CLNT-00096919,00115 02001 80000000068 25,0.000000e+00,2019-06-17,Active,2024-12-31,Jérémy Lacroix,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais
316599,CLNT-00096911,00115 02001 80000000007 52,0.000000e+00,2021-03-23,Active,2024-12-31,Timéo Guillet,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais


In [47]:
dataFrameMerged = dataFrameMerged[dataFrameMerged["AccountStatus"] == "Active"]
dataFrameMerged

,Code,noCompte,AvailableBalance,OpeningDate,AccountStatus,Report_date_to,gestionnaire de compte,BankCode,BranchCode,ProductCode,AccountNumber,productCode,Product,CodeBranch,Branch
0,CLNT-00005640,00115 00001 90000316601 39,1.698926e+09,2024-12-30,Active,2024-12-31,Yanis Blanchard,00115,00001,9,9000031,9,Dépôt à Terme,00001,Champagne-Ardenne
1,CLNT-00022135,00115 00004 90000316600 53,1.322200e+09,2024-12-06,Active,2024-12-31,Sergei Titov,00115,00004,9,9000031,9,Dépôt à Terme,00004,Bourgogne
2,CLNT-00084246,00115 00014 80000316599 27,1.117828e+09,2022-05-23,Active,2024-12-31,Samuel Otieno,00115,00014,8,8000031,8,Compte Epargne,00014,Franche-Comté
3,CLNT-00005629,00115 00001 80000316598 31,1.061136e+09,2023-11-28,Active,2024-12-31,Enzo Giraud,00115,00001,8,8000031,8,Compte Epargne,00001,Champagne-Ardenne
4,CLNT-00005639,00115 00001 90000316597 53,1.000000e+09,2024-12-10,Active,2024-12-31,Enzo Giraud,00115,00001,9,9000031,9,Dépôt à Terme,00001,Champagne-Ardenne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316596,CLNT-00096913,00115 02001 80000000009 46,0.000000e+00,2022-05-04,Active,2024-12-31,Timéo Guillet,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais
316597,CLNT-00096916,00115 02001 80000000012 25,0.000000e+00,2024-02-21,Active,2024-12-31,Timéo Guillet,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais
316598,CLNT-00096919,00115 02001 80000000068 25,0.000000e+00,2019-06-17,Active,2024-12-31,Jérémy Lacroix,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais
316599,CLNT-00096911,00115 02001 80000000007 52,0.000000e+00,2021-03-23,Active,2024-12-31,Timéo Guillet,00115,02001,8,8000000,8,Compte Epargne,02001,Nord-Pas-de-Calais


In [ ]:
numberAccountsByBranch = dataFrameMerged.groupby("Branch").size().reset_index(name="Compte")
numberAccountsByBranch

,Branch,noCompte
0,Alsace,6309
1,Aquitaine,9528
2,Auvergne,7587
3,Basse-Normandie,348
4,Bourgogne,6587
5,Bretagne,7122
6,Centre,2634
7,Champagne-Ardenne,5638
8,Corse,2816
9,Franche-Comté,3521


In [50]:
totalBalanceByBranch = (
    dataFrameMerged.groupby("Branch")["AvailableBalance"]
    .sum()
    .reset_index(name="TotalAvailableBalance")
)
totalBalanceByBranch

,Branch,TotalAvailableBalance
0,Alsace,2.136488e+09
1,Aquitaine,2.741254e+09
2,Auvergne,7.736449e+08
3,Basse-Normandie,2.032248e+09
4,Bourgogne,3.506782e+09
5,Bretagne,1.965211e+09
6,Centre,2.018167e+08
7,Champagne-Ardenne,1.275999e+10
8,Corse,3.405437e+08
9,Franche-Comté,1.652575e+09


In [51]:
averageBalanceByProduct = (
    dataFrameMerged.groupby("ProductCode")["AvailableBalance"]
    .mean()
    .reset_index(name="AverageAvailableBalance")
)
averageBalanceByProduct

,ProductCode,AverageAvailableBalance
0,1,6.378250e+01
1,7,1.775888e+05
2,8,8.991191e+04
3,9,3.913321e+08


In [55]:
averageAccountByProduct = (
    dataFrameMerged.groupby("ProductCode").size().reset_index(name="Compte")
)
averageAccountByProduct

,ProductCode,Compte
0,1,4
1,7,44932
2,8,271578
3,9,19


In [52]:
topGestionnaires = (
    dataFrameMerged.groupby("gestionnaire de compte")["AvailableBalance"]
    .sum()
    .nlargest(10)
    .reset_index(name="TotalBalance")
)
topGestionnaires

,gestionnaire de compte,TotalBalance
0,Enzo Giraud,4.450647e+09
1,Yanis Blanchard,2.549231e+09
2,Léa Caron,1.887348e+09
3,Sergei Titov,1.491131e+09
4,Samuel Otieno,1.251539e+09
5,Tiana Ravelomanana,1.011658e+09
6,Samira Farouk,9.232445e+08
7,Bruno Almeida,9.201887e+08
8,Jérémy Lacroix,7.999519e+08
9,Wang Tao,7.965721e+08


In [54]:
actifAccountNumber = (dataFrameMerged["AccountStatus"] == "Active").sum()
actifAccountNumber

np.int64(316533)